In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import sklearn.linear_model as linear_model

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import  GradientBoostingClassifier,RandomForestClassifier

import lightgbm as lgbm
import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.linear_model import  LinearRegression, Ridge
import os, gc, warnings
warnings.filterwarnings('ignore')

In [154]:
ttrain = pd.read_csv('Train.csv')
ttest = pd.read_csv('Test.csv')
submission = pd.read_csv("SampleSubmission.csv")

In [155]:
fin_data = pd.concat([ttrain,ttest]).reset_index(drop=True)

In [156]:
fin_data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0.0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1.0
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0.0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1.0


In [157]:
# making boolean series for a team name
filter1 = fin_data["State"] =="Lagos"
  
# making boolean series for age
#filter2 = data["Age"]>24
  
# filtering data on basis of both filters
fin_data.where(filter1)

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20.0,2010-08-29,3.0,NaN,NaN,NaN,Lagos,Lagos,Muuve,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13276,ID_ZTTHC5X,2010-12-05,2011-12-04,Male,67.0,2010-12-05,2.0,JEEP,Black,TOYOTA,Victoria Island,Lagos,Car Classic,NaN
13277,ID_ZUJAFUP,2010-01-14,2011-01-13,Male,43.0,2010-01-14,1.0,Saloon,Silver,Hyundai,Surulere,Lagos,Car Classic,NaN
13278,ID_ZWHCTUM,2010-07-26,2011-07-25,Male,30.0,2010-07-26,1.0,Truck,White,Iveco,Victoria Island,Lagos,CVTP,NaN
13279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## A little data processing

In [158]:
pd.crosstab(fin_data.LGA_Name, fin_data.State)

State,ABULE-EGBA,AJAO-ESTATE,Aba-North,Aba-South,Abia,Aboh-Mbaise,Abuja,Abuja-Municipal,Ado-Ekiti,Ado-Ota,...,Oyo-West,Port-Harcourt,QuaAn-Pan,Rivers,Udi-Agwu,Ughelli-North,Umuahia-South,Warri-Central,Warri-North,Warri-South
LGA_Name,,,,,,,,,,,,,,,,,,,,,
IFAKO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABULE EGBA,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AGBARA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AJAO ESTATE,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AKOKO-EDO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yenagoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yorro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zaria,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
## There are too many zeros

In [160]:
fin_data.drop(['ID','LGA_Name'], axis =1, inplace = True)

In [161]:
fin_data.head(10)

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,State,ProductName,target
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,Car Classic,0.0
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,Car Classic,1.0
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,Car Classic,0.0
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,CarSafe,0.0
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Muuve,1.0
5,2010-10-21,2011-10-20,Male,37,2010-10-21,2,NaN,NaN,NaN,NaN,Car Classic,0.0
6,2010-08-02,2011-08-01,Female,40,2010-08-02,3,NaN,NaN,NaN,Lagos,CVTP,0.0
7,2010-09-12,2011-09-11,Male,43,2010-09-12,1,Saloon,NaN,TOYOTA,NaN,CarSafe,0.0
8,2010-07-06,2011-07-05,Female,71,2010-07-06,2,JEEP,Black,REXTON,Benue,Car Classic,0.0
9,2010-12-18,2011-12-17,Female,58,2010-12-18,1,NaN,NaN,NaN,Eti-Osa,Car Classic,0.0


In [162]:
import re
all_data = fin_data.rename(columns = lambda x:re.sub('[*=/,&^#@]+', '', x)) 

## DEALING WITH MISSING VALUES

In [163]:
fin_data.isnull().sum()

Policy Start Date            0
Policy End Date              0
Gender                     400
Age                          0
First Transaction Date       0
No_Pol                       0
Car_Category              4110
Subject_Car_Colour        7659
Subject_Car_Make          2724
State                     7144
ProductName                  0
target                    1202
dtype: int64

In [164]:
list = []
for i in fin_data.columns:
    kd = fin_data[i].isna().sum()
    ml = len(fin_data[i])
    jd = kd/ml * 100
    
    
    if jd > 70:
        fin_data.drop(i, axis = 1, inplace = True)
        list.append(i)
        
        
        """ Dropping fetaures with missing values greater than 70% """

In [165]:
#fin_data['age'].fillna(fin_data['age'].median(), inplace = True) ## replaced the missing value in age with the median
## median is robust to outliers.
fin_data['Gender'].value_counts()

Male            8356
Female          3679
Entity           300
Joint Gender     238
NOT STATED       190
NO GENDER         76
SEX               42
Name: Gender, dtype: int64

In [166]:
pd.crosstab(fin_data['Gender'], fin_data['target'])

target,0.0,1.0
Gender,,
Entity,183,94
Female,2969,358
Joint Gender,201,22
Male,6715,902
NO GENDER,53,13
NOT STATED,169,6
SEX,35,0


In [167]:
mapper = {'Joint Gender':'Other', 'NOT STATED':'Other', 'NO GENDER': 'Other', 'SEX':"Other"}
fin_data.Gender = fin_data.Gender.replace(mapper)

# Confirm mappings
fin_data.Gender.value_counts()

Male      8356
Female    3679
Other      546
Entity     300
Name: Gender, dtype: int64

In [168]:
fin_data['State'].unique()

array([nan, 'Lagos', 'Benue', 'Eti-Osa', 'Delta', 'Ogun',
       'Abuja-Municipal', 'Oshodi-Isolo', 'Ibeju-Lekki', 'Ibadan-West',
       'Obia-Akpor', 'Ibadan-East', 'Kaduna-South', 'Amuwo-Odofin',
       'Anambra', 'Rivers', 'Niger-State', 'Edo', 'Akwa-Ibom',
       'ENUGU-EAST', 'Abuja', 'Kaduna', 'Esan-West', 'Obafemi-Owode',
       'Orile-Iganmu', 'Ifako-Ijaye', 'Ibadan-North', 'Imo', 'Jos-North',
       'Ondo-West', 'Kaduna-North', 'Aniocha-South', 'Awka-South',
       'Ile-Oluji', 'Ijebu-Ode', 'Port-Harcourt', 'Nnewi-North',
       'Aboh-Mbaise', 'Oyo', 'Akoko-West', 'Warri-Central', 'Cross-River',
       'Jos-South', 'Onitsha-North', 'Osun', 'Ajeromi-Ifelodun',
       'Kano-Municipal', 'Ife-Central', 'Ilorin-West', 'Lagelu-North',
       'Owerri-Municipal', 'Nsit-Ubium', 'Udi-Agwu', 'Essien-Udim',
       'Owerri-West', 'Ondo', 'Ogbmosho-South', 'Umuahia-South',
       'ABULE-EGBA', 'Enugu-North', 'Ovia-SouthWest', 'Kebbi',
       'Ajegunle-State', 'Ekiti-West', 'Ogun-Waterside',

In [169]:
pd.crosstab(fin_data['State']=='', fin_data['target'])

target,0.0,1.0
State,,
False,10624,1455


In [170]:
## Mapping  by geopolitical zones

mapper = {'Ibeju-Lekki':'Geo1','Oshodi-Isolo':'Geo1','ABULE-EGBA':'Geo1','AJAO-ESTATE':'Geo1','Orile-Iganmu':'Geo1',
          'Ajeromi-Ifelodun':'Geo1','Ifako-Ijaye':'Geo1','Amuwo-Odofin':'Geo1','Eti-Osa':'Geo1','Ajegunle-State':'Geo1',
          'Ibadan-East':'Geo1','Ibadan-West':'Geo1', 'Ibadan-North':'Geo1','Oyo-East':'Geo1','Oyo':'Geo1',
          'Ogbmosho-South':'Geo1','Oyo-West':'Geo1', 'Lagelu-North':'Geo1','Ibarapa-Central':'Geo1','Lagos':'Geo1',
          'Ife-Central':'Geo1', 'Ife-North':'Geo1','Ilesha-West':'Geo1','Ilesha-East':'Geo1','Osun':'Geo1',
          'Ile-Oluji':'Geo1','Ondo_West':'Geo1','Ondo':'Geo1',
          'Obafemi-Owode':'Geo1','Ogun-Waterside':'Geo1','Ogun-Waterside':'Geo1','Ijebu-North':'Geo1','Ijebu-Ode':'Geo1',
          'Ijebu-East':'Geo1', 'Ado-Ota':'Geo1', 'Ogun':'Geo1',
          'Ado-Ekiti':'Geo1', 'Ekiti-East':'Geo1','Akoko-West':'Geo1','Ondo-West':'Geo1','Ekiti-West':'Geo1',
          
          
          'Onitsha-North':'Geo2', 'Onitsha-South':'Geo2','Anambra-East':'Geo2','Awka-North':'Geo2',
          'Awka-South':'Geo2','Idemili-south':'Geo2','Nnewi-South':'Geo2','Idemili-North':'Geo2','Anambra':'Geo2',
          'Ngor-Okpala':'Geo2','Owerri-North':'Geo2','Owerri-West':'Geo2','Owerri-Municipal':'Geo2',
          'Owerri-West':'Geo2','Imo':'Geo2','Aboh-Mbaise':'Geo2',
          'Umuahia-South':'Geo2', 'Aba-North':'Geo2','Aba-South':'Geo2','Abia':'Geo2','Udi-Agwu':'Geo2',
          'Enugu-North':'Geo2','ENUGU-EAST':'Geo2','ENUGU-SOUTH':'Geo2','Nnewi-North':'Geo2','Ebonyi':'Geo2',
          
          
          'Essien-Udim':'Geo3','Nsit-Ubium':'Geo3','Akwa-Ibom':'Geo3',
          'Obia-Akpor':'Geo3','Port-Harcourt':'Geo3','Asari-Toru':'Geo3','Calabar-Municipality':'Geo3',
          'Rivers':'Geo3','Cross-River':'Geo3', 'Ovia-SouthWest':'Geo3','Ogba-Ndoni':'Geo3',
          'Warri':'Geo3','Warri-Central':'Geo3','Warri-South':'Geo3','Oshimili-North':'Geo3',
          'Ethiope-East':'Geo3','Ughelli-North':'Geo3', 'Ndokwa-East':'Geo3','Isoko-south':'Geo3',
          'Isoko-North':'Geo3','Warri-North':'Geo3','Delta':'Geo3','Esan-Central':'Geo3','Aniocha-South':'Geo3',
          'Bayelsa':'Geo3','Esan-West':'Geo3','Edo':'Geo3','Etsako-West':'Geo3',
          
          
          'Abuja-Municipal':'Geo4','Central-Abuja':'Geo4','Abuja':'Geo4','QuaAn-Pan':'Geo4',
          'Ilorin-West':'Geo4','Ilorin-East':'Geo4','Kwara':'Geo4','Benue':'Geo4','Nasarawa':'Geo4','Kogi':'Geo4',
          'Niger':'Geo4','Jos-North':'Geo4','Jos-South':'Geo4','Niger-State':'Geo4',
          
          
          'Kaduna-South':'Geo5','Kaduna':'Geo5','Kano':'Geo5','Kaduna-North':'Geo5','Kano-Municipal':'Geo5','Kebbi':'Geo5',
          
          
          'Bauchi':'Geo6','Gombe':'Geo6',
          
          
          'N-A':'None'}

fin_data.State = fin_data.State.replace(mapper)

# Confirm mappings
fin_data.State.value_counts()

Geo1    4377
Geo4    1083
Geo3     451
Geo2     164
Geo5      57
Geo6       4
None       1
Name: State, dtype: int64

In [171]:
pd.crosstab(fin_data.State, fin_data.target)

target,0.0,1.0
State,,
Geo1,3338,661
Geo2,134,11
Geo3,329,82
Geo4,789,190
Geo5,49,4
Geo6,2,1
None,1,0


In [172]:
## Geopilitical zones 2,5,6 are barely represented. Let's make them into just one

mapper = {'Geo5':'Geo2','Geo6':'Geo2'}
fin_data.State = fin_data.State.replace(mapper)

# Confirm mappings
fin_data.State.value_counts()

Geo1    4377
Geo4    1083
Geo3     451
Geo2     225
None       1
Name: State, dtype: int64

In [173]:
## Geopilitical zones 2,5,6 are barely represented. Let's make them into just one

mapper = {'Geo2':'Geo4','Geo3':'Geo4'}
fin_data.State = fin_data.State.replace(mapper)

# Confirm mappings
fin_data.State.value_counts()

Geo1    4377
Geo4    1759
None       1
Name: State, dtype: int64

In [174]:
def fill(col):
    for i in col:
        #fin_data[i].fillna(fin_data[i].mode()[0], inplace = True)
        #fin_data[i].astype(str)
        fin_data[i].fillna(fin_data[i].mode()[0], inplace = True)
        
        """ Filling up the remaining features with a very large number """

In [175]:
fill(fin_data.drop(["target","State",'Car_Category'],axis=1))

In [176]:
fin_data["State"].fillna('None', inplace = True)
fin_data["Car_Category"].fillna('None', inplace = True)

In [177]:
fin_data['State'].unique()

array(['None', 'Geo1', 'Geo4'], dtype=object)

In [178]:
dcol = [col for col in fin_data.columns if 'Date' in col]

In [179]:
for col in dcol:
    fin_data[col] = pd.to_datetime(fin_data[col])

In [180]:
for col in dcol:
  for date_feature in ['year', 'month', 'day']:
    fin_data[col+date_feature] = getattr(fin_data[col].dt, date_feature)

fin_data.head()

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,State,...,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,None,...,0.0,2010,5,14,2011,5,13,2010,5,14
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,None,...,1.0,2010,11,29,2011,11,28,2010,11,29
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,None,...,0.0,2010,3,21,2011,3,20,2010,3,21
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,None,Black,TOYOTA,None,...,0.0,2010,8,21,2011,8,20,2010,8,21
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,None,Black,TOYOTA,Geo1,...,1.0,2010,8,29,2010,12,31,2010,8,29


In [181]:
#pd.crosstab(fin_data['Policy Start Dateyear'], fin_data['First Transaction Dateyear'])

In [182]:
## Too many of the dates are in 2010, so it's advisable to drop both columns

In [183]:
pd.crosstab(fin_data['Policy Start Dateyear'], fin_data['Policy End Dateyear'])

## better to find differnce between the dates because using the years would not help all that much

Policy End Dateyear,2010,2011,2012,2016,2034,2037,2041
Policy Start Dateyear,,,,,,,
2001,0,1,0,0,0,0,0
2002,0,1,0,0,0,0,0
2003,0,1,0,0,1,0,0
2005,0,1,0,0,0,0,0
2006,0,1,0,1,0,0,0
2007,0,0,0,0,0,1,0
2009,0,5,0,1,0,0,0
2010,177,13086,2,0,0,0,2


In [184]:
pd.crosstab(fin_data['Policy End Dateyear'], fin_data['First Transaction Dateyear'])

## finally, we drop the three year features

First Transaction Dateyear,2001,2002,2003,2005,2006,2007,2009,2010
Policy End Dateyear,,,,,,,,
2010,0,0,0,0,0,0,0,177
2011,1,1,1,1,1,0,5,13086
2012,0,0,0,0,0,0,0,2
2016,0,0,0,0,1,0,1,0
2034,0,0,1,0,0,0,0,0
2037,0,0,0,0,0,1,0,0
2041,0,0,0,0,0,0,0,2


In [185]:
pd.crosstab(fin_data['Policy Start Dateday'], fin_data['First Transaction Dateday'])

First Transaction Dateday,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
Policy Start Dateday,,,,,,,,,,,,,,,,,,,,,
1,430,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,398,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,351,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,446,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,458,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,432,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,431,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,445,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,409,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
""" 
    drop either fin_data['First Transaction Datemonth']) or fin_data['Policy Start Datemonth']
    and
    drop either fin_data['First Transaction Dateday']) or fin_data['Policy Start Dateday'] due to very high correlation
"""


" \n    drop either fin_data['First Transaction Datemonth']) or fin_data['Policy Start Datemonth']\n    and\n    drop either fin_data['First Transaction Dateday']) or fin_data['Policy Start Dateday'] due to very high correlation\n"

In [187]:
fin_data.head()

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,State,...,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,None,...,0.0,2010,5,14,2011,5,13,2010,5,14
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,None,...,1.0,2010,11,29,2011,11,28,2010,11,29
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,None,...,0.0,2010,3,21,2011,3,20,2010,3,21
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,None,Black,TOYOTA,None,...,0.0,2010,8,21,2011,8,20,2010,8,21
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,None,Black,TOYOTA,Geo1,...,1.0,2010,8,29,2010,12,31,2010,8,29


In [188]:
fin_data["Policy_Month_Diff"] = abs(fin_data['Policy End Datemonth'] - fin_data['Policy Start Datemonth'])
fin_data['Policy_Day_Diff'] = abs(fin_data['Policy End Date'] - fin_data['Policy Start Date']).dt.days
fin_data['Policy_Week_Diff'] = abs(round(fin_data['Policy_Day_Diff']/7, 0))

## for most of the features, the time period falls in the same year. So, taking the monthly difference makes sense.
## finding the number of working weeks might also help our model. We can drop the policy days features to avoid multicollinearity.

In [189]:
fin_data.drop(['Policy Start Date','Policy End Date','First Transaction Date'], axis =1, inplace = True)
fin_data.drop(['Policy Start Dateyear','Policy End Dateyear','First Transaction Dateyear'], axis =1, inplace = True)
fin_data.drop(['First Transaction Datemonth'], axis =1, inplace = True)
fin_data.drop(['First Transaction Dateday'], axis =1, inplace = True)

In [190]:
fin_data.drop(['Policy_Day_Diff'], axis =1, inplace = True)

In [191]:
## drop policy end date day and policy end datemonth since we have taken the difference already
fin_data.drop(['Policy End Dateday','Policy End Datemonth'], axis =1, inplace = True)

In [192]:
fin_data.corr()

,Age,No_Pol,target,Policy Start Datemonth,Policy Start Dateday,Policy_Month_Diff,Policy_Week_Diff
Age,1.000000,0.018007,-0.028937,-0.015398,0.009114,-0.019879,0.007290
No_Pol,0.018007,1.000000,0.074245,-0.021703,-0.037983,0.123153,-0.019761
target,-0.028937,0.074245,1.000000,-0.072584,-0.019590,0.070779,-0.022787
Policy Start Datemonth,-0.015398,-0.021703,-0.072584,1.000000,-0.004020,0.218722,-0.074350
Policy Start Dateday,0.009114,-0.037983,-0.019590,-0.004020,1.000000,-0.093216,0.003956
Policy_Month_Diff,-0.019879,0.123153,0.070779,0.218722,-0.093216,1.000000,-0.210570
Policy_Week_Diff,0.007290,-0.019761,-0.022787,-0.074350,0.003956,-0.210570,1.000000


## Dealing with the age feature

In [193]:
pd.crosstab(fin_data['Age'], fin_data['target'])

target,0.0,1.0
Age,,
-6099,1,0
-5939,0,2
-76,3,0
-51,1,0
-27,0,1
...,...,...
128,1,0
133,1,0
140,1,0


In [194]:
listt = []
for i in range(len(fin_data['Age'])):
    if fin_data['Age'][i] < 0:
        fin_data['Age'][i] = abs(fin_data['Age'][i])

In [195]:
fin_data['Age'][fin_data['Age']==6099] = fin_data['Age'].mode()[0]
fin_data['Age'][fin_data['Age']==5939] = fin_data['Age'].mode()[0]

In [196]:
fin_data['Age'].value_counts()

40     903
50     686
41     550
36     537
38     514
      ... 
90       1
17       1
128      1
112      1
88       1
Name: Age, Length: 101, dtype: int64

## REMOVING SKEWNESS AND OUTLIERS

In [197]:
numerical = fin_data.drop(['target'], axis =1).dtypes[fin_data.drop(['target'], axis =1).dtypes != 'object'].index

skewness = fin_data.drop(['target'], axis =1)[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness_df = pd.DataFrame({'Skewness': skewness})
skewness_df.head(10)

,Skewness
Policy_Week_Diff,44.518024
No_Pol,3.772737
Age,3.278167
Policy_Month_Diff,3.210912
Policy Start Datemonth,0.030754
Policy Start Dateday,-0.047862


In [198]:
fin_data.Policy_Week_Diff.unique()

array([  52.,   18.,   50.,   46.,   26.,   51.,   17.,   48.,   47.,
         31.,   43.,   39.,   45.,   15.,   34.,   49.,   25.,   22.,
         35.,   38.,   13.,   27.,  104.,   14.,   33.,   19.,   41.,
         37.,   36.,    4.,   32.,   44.,   29.,    2.,   42.,    3.,
         20., 1591.,   28.,   40.,   10., 1587.,   23.,   53.,   12.,
         30.,   56.,   16.,   74.,   21.,    9.,   24.,    7.,  382.,
          5.,    6.,  531.,   54.,   72.,   63.,  522.,  321.,   78.,
        417.,  469.,    8.,  266.,   55.])

In [199]:
from scipy.special import boxcox1p

boxcox_features = skewness_df[np.abs(skewness_df['Skewness'])>0.75].index
lam = 0.25

for col in boxcox_features:
    fin_data.loc[:, col] = boxcox1p(fin_data[col], lam)

In [200]:
# from scipy.stats import norm, skew, probplot

# fin_data['Policy_Week_Diff'] = np.log1p(fin_data['Policy_Week_Diff'])

#mu, sigma = norm.fit(fin_data['Policy_Week_Diff'])
#sns.distplot(fin_data['Policy_Week_Diff'], fit=norm)
#plt.legend(['Normal distribution ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc = 'best')
#plt.ylabel('Frequency')
#plt.title('Histogram of SalePrice');

#fig = plt.figure()
#res = probplot(fin_data['Policy_Week_Diff'], plot=plt);

In [201]:
numerical = fin_data.dtypes[fin_data.dtypes != 'object'].index

skewness = fin_data[numerical].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness_df = pd.DataFrame({'Skewness': skewness})
skewness_df.head(10)

,Skewness
Policy_Month_Diff,2.623812
No_Pol,2.484713
target,2.332096
Policy_Week_Diff,0.550963
Policy Start Datemonth,0.030754
Policy Start Dateday,-0.047862
Age,-1.356334


In [202]:
pd.crosstab(fin_data['Subject_Car_Colour'], fin_data['target'])

target,0.0,1.0
Subject_Car_Colour,,
As Attached,381,117
Ash,104,29
B.Silver,2,0
Beige Mitalic,1,0
Black,8122,720
Black & Orange,2,0
Black & White,1,1
Blue,260,101
Blue & Red,1,2


In [203]:
## I dont beliove car color is important to our predicton, so I am dropping it.

In [204]:
fin_data.drop(['Subject_Car_Colour'], axis =1, inplace = True)

In [205]:
pd.crosstab(fin_data['Subject_Car_Make'], fin_data['target'])

target,0.0,1.0
Subject_Car_Make,,
.,30,14
ABG,1,0
ACURA,58,7
As Attached,6,5
Ashok Leyland,1,0
...,...,...
Volkswagen,117,5
Volvo,29,8
Wrangler Jeep,2,0


In [206]:
fin_data.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,State,ProductName,target,Policy Start Datemonth,Policy Start Dateday,Policy_Month_Diff,Policy_Week_Diff
0,Male,5.438444,0.756828,Saloon,TOYOTA,None,Car Classic,0.0,5,14,0.000000,6.792672
1,Female,7.962790,0.756828,JEEP,TOYOTA,None,Car Classic,1.0,11,29,0.000000,6.792672
2,Male,6.302038,0.756828,Saloon,TOYOTA,None,Car Classic,0.0,3,21,0.000000,6.792672
3,Male,1.264296,0.756828,None,TOYOTA,None,CarSafe,0.0,8,21,0.000000,6.792672
4,Entity,4.562781,1.656854,None,TOYOTA,Geo1,Muuve,1.0,8,29,1.981395,4.351191


In [207]:
pd.crosstab(fin_data['Car_Category'], fin_data['target'])

target,0.0,1.0
Car_Category,,
Bus,45,9
CAMRY CAR HIRE,1,0
JEEP,1773,248
Mini Bus,26,10
Mini Van,11,2
Motorcycle,15,2
None,3410,328
Pick Up,22,8
Pick Up > 3 Tons,1,0


In [208]:
fin_data['Car_Category'].unique()

array(['Saloon', 'JEEP', 'None', 'Motorcycle', 'Truck', 'Bus', 'Mini Bus',
       'Pick Up', 'Mini Van', 'Van', 'Pick Up > 3 Tons', 'CAMRY CAR HIRE',
       'Wagon', 'Shape Of Vehicle Chasis', 'Sedan', 'Station 4 Wheel',
       'Tipper Truck'], dtype=object)

In [209]:
fin_data['Car_Category'].unique().shape

(17,)

In [210]:
fin_data['Subject_Car_Make'].unique()

array(['TOYOTA', 'REXTON', 'Lexus', 'Hyundai', 'Iveco', 'DAF', 'Honda',
       'Mercedes', 'Jincheng', 'ACURA', 'Ford', 'Volkswagen', 'Nissan',
       '.', 'Pontiac', 'Range Rover', 'Kia', 'Mitsubishi', 'Scania',
       'BMW', 'Infiniti', 'Renault', 'Volvo', 'Hummer', 'Mack',
       'Grand Cherokee', 'Porsche', 'Peugeot', 'Land Rover', 'Chevrolet',
       'Mazda', 'Man', 'Jeep', 'Audi', 'Suzuki', 'MINI COOPER', 'KA',
       'As Attached', 'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler',
       'Dodge', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Astra', 'Motorcycle', 'ZOYTE', 'Seat', 'FOTON', 'Howo', 'Lincoln',
       'Rols Royce', 'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black',
       'Tata', 'Ashok Leyland', 'Geely', 'Caddillac', 'Wrangler Jeep',
       'Raston', 'MG', 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj',
       'Datsun'], dtype=object)

In [211]:
fin_data['Subject_Car_Make'].unique().shape

(75,)

In [212]:
## Subject_Car_Make has too many underrepresented values, so we drop it.
fin_data.drop(['Subject_Car_Make'], axis=1, inplace = True)

In [213]:
"""The car_category also has too many underrepresented values, so we regroup it.
""" 

'The car_category also has too many underrepresented values, so we regroup it.\n'

In [214]:
fin_data.Car_Category.value_counts()

Saloon                     6633
None                       4110
JEEP                       2223
Truck                       108
Bus                          56
Mini Bus                     45
Pick Up                      32
Motorcycle                   18
Sedan                        14
Mini Van                     13
Wagon                        10
Station 4 Wheel               6
Shape Of Vehicle Chasis       6
Van                           3
Pick Up > 3 Tons              2
Tipper Truck                  1
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [215]:
"""Regrouping the Car_Category feature"""

mapper = {'Mini Van':'Other','Pick Up':'Other','Sedan':'Other','Mini Van':'Other',
          'Tipper Truck':'Other',"Van":'Other',"Bus":'Other',"Truck":"Other",
         "Mini Bus":'Other', "Wagon": 'Other',"Shape Of Vehicle Chasis":'Other', 
         "Station 4 Wheel": 'Other', "Pick Up > 3 Tons": 'Other', "CAMRY CAR HIRE": 'Other',"Motorcycle": 'Other'}
fin_data.Car_Category = fin_data.Car_Category.replace(mapper)

# Confirm mappings
fin_data.Car_Category.value_counts()

Saloon    6633
None      4110
JEEP      2223
Other      315
Name: Car_Category, dtype: int64

In [216]:
pd.crosstab(fin_data['target'], fin_data['Car_Category'])

Car_Category,JEEP,None,Other,Saloon
target,,,,
0.0,1773,3410,222,5219
1.0,248,328,64,815


In [217]:
#pd.crosstab(fin_data['Gender'], fin_data['target'])

In [218]:
## intution: join the "other" and "none" category together

mapper = {"None": 'Other'}
fin_data.Car_Category = fin_data.Car_Category.replace(mapper)

# Confirm mappings
fin_data.Car_Category.value_counts()

Saloon    6633
Other     4425
JEEP      2223
Name: Car_Category, dtype: int64

In [219]:
fin_data.head()

,Gender,Age,No_Pol,Car_Category,State,ProductName,target,Policy Start Datemonth,Policy Start Dateday,Policy_Month_Diff,Policy_Week_Diff
0,Male,5.438444,0.756828,Saloon,None,Car Classic,0.0,5,14,0.000000,6.792672
1,Female,7.962790,0.756828,JEEP,None,Car Classic,1.0,11,29,0.000000,6.792672
2,Male,6.302038,0.756828,Saloon,None,Car Classic,0.0,3,21,0.000000,6.792672
3,Male,1.264296,0.756828,Other,None,CarSafe,0.0,8,21,0.000000,6.792672
4,Entity,4.562781,1.656854,Other,Geo1,Muuve,1.0,8,29,1.981395,4.351191


In [220]:
fin_data.ProductName.value_counts()

Car Classic         7142
CarSafe             4154
Customized Motor     605
Car Plus             523
CVTP                 509
CarFlex              194
Muuve                100
Motor Cycle           49
Car Vintage            5
Name: ProductName, dtype: int64

In [221]:
mapper = {"Car Vintage": 'Car Classic',"Motor Cycle": "Customized Motor"}
fin_data.ProductName = fin_data.ProductName.replace(mapper)

# Confirm mappings
fin_data.ProductName.value_counts()

Car Classic         7147
CarSafe             4154
Customized Motor     654
Car Plus             523
CVTP                 509
CarFlex              194
Muuve                100
Name: ProductName, dtype: int64

In [222]:
mapper = {"Customized Motor": 'Other', "Car Plus": "Other", "CVTP": 'Other',"CarFlex": "Other", "Muuve": 'Other'}
fin_data.ProductName = fin_data.ProductName.replace(mapper)

# Confirm mappings
fin_data.ProductName.value_counts()

Car Classic    7147
CarSafe        4154
Other          1980
Name: ProductName, dtype: int64

In [223]:
pd.crosstab(fin_data.ProductName, fin_data.target)

target,0.0,1.0
ProductName,,
Car Classic,5416,1074
CarSafe,3760,14
Other,1448,367


In [224]:
fin_data.isna().sum()

Gender                       0
Age                          0
No_Pol                       0
Car_Category                 0
State                        0
ProductName                  0
target                    1202
Policy Start Datemonth       0
Policy Start Dateday         0
Policy_Month_Diff            0
Policy_Week_Diff             0
dtype: int64

## ENCODING CATEGORICAL FEATURES

In [225]:
# Label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Create new features
#fin_data['ProductName'] = le.fit_transform(fin_data['ProductName'])
#fin_data['Car_Category'] = le.fit_transform(fin_data['Car_Category'])
#fin_data['State'] = le.fit_transform(fin_data['State'])
#fin_data['Gender'] = le.fit_transform(fin_data['State'])

In [226]:
from sklearn.model_selection import StratifiedKFold

In [227]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=99)

In [228]:
split=ttrain.shape[0]
train_df=fin_data[:split]
test_df=fin_data[split:]

In [229]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
    
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [230]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits = 5, shuffle = True, random_state=99)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [231]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [232]:
train_feat, test_feat = mean_target_encoding(train_df, test_df, 'target', 'Car_Category', alpha=5)

In [233]:
train_df['Car_Category'] = train_feat
test_df['Car_Category'] = test_feat

In [234]:
train_feat2, test_feat2 = mean_target_encoding(train_df, test_df, 'target', 'State', alpha=5)

In [235]:
train_df['State'] = train_feat2
test_df['State'] = test_feat2

In [236]:
train_feat3, test_feat3 = mean_target_encoding(train_df, test_df, 'target', 'ProductName', alpha=5)

In [237]:
train_df['ProductName'] = train_feat3
test_df['ProductName'] = test_feat3

In [238]:
train_feat4, test_feat4 = mean_target_encoding(train_df, test_df, 'target', 'Gender', alpha=5)

In [239]:
train_df['Gender'] = train_feat4
test_df['Gender'] = test_feat4

In [240]:
test_df.isna().sum()

Gender                       0
Age                          0
No_Pol                       0
Car_Category                 0
State                        0
ProductName                  0
target                    1202
Policy Start Datemonth       0
Policy Start Dateday         0
Policy_Month_Diff            0
Policy_Week_Diff             0
dtype: int64

In [241]:
#test_df = test_df.drop(['target'], axis =1, inplace = True)

In [242]:
X = train_df.drop(['target'], axis =1)
y = train_df['target']

In [243]:
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler()
#X, y = ros.fit_resample(X, y)

from imblearn.over_sampling import SMOTE
over = SMOTE()
X, y = over.fit_resample(X, y)


from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import BorderlineSMOTE

In [244]:
#!pip install imblearn

In [4]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=19)

NameError: name 'X' is not defined

In [246]:
from catboost import Pool
cat_model= CatBoostClassifier(silent=True, eval_metric = 'AUC')
cat_model= CatBoostClassifier()
cat_model.fit(Pool(X_train,y_train))
cat = cat_model.predict(X_test)

Learning rate set to 0.032625
0:	learn: 0.6647348	total: 23.9ms	remaining: 23.9s
1:	learn: 0.6427403	total: 35.1ms	remaining: 17.5s
2:	learn: 0.6190949	total: 46ms	remaining: 15.3s
3:	learn: 0.6041471	total: 56.8ms	remaining: 14.1s
4:	learn: 0.5838324	total: 67.5ms	remaining: 13.4s
5:	learn: 0.5690742	total: 78.5ms	remaining: 13s
6:	learn: 0.5590893	total: 89.5ms	remaining: 12.7s
7:	learn: 0.5439319	total: 101ms	remaining: 12.5s
8:	learn: 0.5273588	total: 112ms	remaining: 12.3s
9:	learn: 0.5143669	total: 122ms	remaining: 12.1s
10:	learn: 0.5057971	total: 133ms	remaining: 12s
11:	learn: 0.4975183	total: 146ms	remaining: 12.1s
12:	learn: 0.4844134	total: 157ms	remaining: 11.9s
13:	learn: 0.4783417	total: 168ms	remaining: 11.9s
14:	learn: 0.4654401	total: 179ms	remaining: 11.8s
15:	learn: 0.4606698	total: 192ms	remaining: 11.8s
16:	learn: 0.4558520	total: 210ms	remaining: 12.1s
17:	learn: 0.4476899	total: 233ms	remaining: 12.7s
18:	learn: 0.4419824	total: 244ms	remaining: 12.6s
19:	learn:

170:	learn: 0.2459815	total: 2.16s	remaining: 10.5s
171:	learn: 0.2456975	total: 2.17s	remaining: 10.4s
172:	learn: 0.2452300	total: 2.19s	remaining: 10.5s
173:	learn: 0.2450160	total: 2.2s	remaining: 10.4s
174:	learn: 0.2442902	total: 2.21s	remaining: 10.4s
175:	learn: 0.2439558	total: 2.23s	remaining: 10.4s
176:	learn: 0.2429665	total: 2.24s	remaining: 10.4s
177:	learn: 0.2428354	total: 2.25s	remaining: 10.4s
178:	learn: 0.2424347	total: 2.26s	remaining: 10.4s
179:	learn: 0.2420705	total: 2.27s	remaining: 10.4s
180:	learn: 0.2416528	total: 2.29s	remaining: 10.3s
181:	learn: 0.2414879	total: 2.3s	remaining: 10.3s
182:	learn: 0.2412029	total: 2.31s	remaining: 10.3s
183:	learn: 0.2410139	total: 2.32s	remaining: 10.3s
184:	learn: 0.2407003	total: 2.34s	remaining: 10.3s
185:	learn: 0.2397185	total: 2.35s	remaining: 10.3s
186:	learn: 0.2395758	total: 2.36s	remaining: 10.3s
187:	learn: 0.2392165	total: 2.38s	remaining: 10.3s
188:	learn: 0.2389331	total: 2.39s	remaining: 10.3s
189:	learn: 0.

334:	learn: 0.2043547	total: 4.37s	remaining: 8.67s
335:	learn: 0.2041423	total: 4.38s	remaining: 8.66s
336:	learn: 0.2040113	total: 4.39s	remaining: 8.64s
337:	learn: 0.2037913	total: 4.4s	remaining: 8.62s
338:	learn: 0.2035708	total: 4.41s	remaining: 8.6s
339:	learn: 0.2034565	total: 4.42s	remaining: 8.59s
340:	learn: 0.2031848	total: 4.44s	remaining: 8.57s
341:	learn: 0.2028585	total: 4.45s	remaining: 8.56s
342:	learn: 0.2026551	total: 4.46s	remaining: 8.54s
343:	learn: 0.2025626	total: 4.47s	remaining: 8.53s
344:	learn: 0.2022822	total: 4.49s	remaining: 8.52s
345:	learn: 0.2021564	total: 4.5s	remaining: 8.5s
346:	learn: 0.2017959	total: 4.51s	remaining: 8.49s
347:	learn: 0.2012904	total: 4.52s	remaining: 8.47s
348:	learn: 0.2011040	total: 4.53s	remaining: 8.45s
349:	learn: 0.2009914	total: 4.54s	remaining: 8.44s
350:	learn: 0.2008766	total: 4.55s	remaining: 8.42s
351:	learn: 0.2007093	total: 4.57s	remaining: 8.41s
352:	learn: 0.2005635	total: 4.58s	remaining: 8.4s
353:	learn: 0.200

495:	learn: 0.1791468	total: 6.32s	remaining: 6.42s
496:	learn: 0.1790204	total: 6.33s	remaining: 6.4s
497:	learn: 0.1787124	total: 6.34s	remaining: 6.39s
498:	learn: 0.1785335	total: 6.35s	remaining: 6.38s
499:	learn: 0.1783930	total: 6.36s	remaining: 6.36s
500:	learn: 0.1782735	total: 6.37s	remaining: 6.35s
501:	learn: 0.1781882	total: 6.38s	remaining: 6.33s
502:	learn: 0.1780916	total: 6.4s	remaining: 6.32s
503:	learn: 0.1780172	total: 6.41s	remaining: 6.31s
504:	learn: 0.1778967	total: 6.42s	remaining: 6.29s
505:	learn: 0.1777904	total: 6.43s	remaining: 6.28s
506:	learn: 0.1776174	total: 6.46s	remaining: 6.28s
507:	learn: 0.1775242	total: 6.47s	remaining: 6.27s
508:	learn: 0.1773892	total: 6.49s	remaining: 6.26s
509:	learn: 0.1773037	total: 6.52s	remaining: 6.26s
510:	learn: 0.1771926	total: 6.53s	remaining: 6.25s
511:	learn: 0.1770507	total: 6.54s	remaining: 6.24s
512:	learn: 0.1769094	total: 6.55s	remaining: 6.22s
513:	learn: 0.1767872	total: 6.57s	remaining: 6.21s
514:	learn: 0.

658:	learn: 0.1632813	total: 8.46s	remaining: 4.38s
659:	learn: 0.1632269	total: 8.47s	remaining: 4.36s
660:	learn: 0.1631682	total: 8.48s	remaining: 4.35s
661:	learn: 0.1631390	total: 8.5s	remaining: 4.34s
662:	learn: 0.1630742	total: 8.51s	remaining: 4.33s
663:	learn: 0.1630057	total: 8.52s	remaining: 4.31s
664:	learn: 0.1629199	total: 8.53s	remaining: 4.3s
665:	learn: 0.1628428	total: 8.54s	remaining: 4.28s
666:	learn: 0.1627572	total: 8.55s	remaining: 4.27s
667:	learn: 0.1626522	total: 8.56s	remaining: 4.26s
668:	learn: 0.1625372	total: 8.58s	remaining: 4.24s
669:	learn: 0.1624839	total: 8.59s	remaining: 4.23s
670:	learn: 0.1624254	total: 8.6s	remaining: 4.22s
671:	learn: 0.1623482	total: 8.62s	remaining: 4.21s
672:	learn: 0.1622904	total: 8.63s	remaining: 4.19s
673:	learn: 0.1622419	total: 8.64s	remaining: 4.18s
674:	learn: 0.1621702	total: 8.66s	remaining: 4.17s
675:	learn: 0.1620799	total: 8.67s	remaining: 4.16s
676:	learn: 0.1619974	total: 8.68s	remaining: 4.14s
677:	learn: 0.1

821:	learn: 0.1515553	total: 10.4s	remaining: 2.26s
822:	learn: 0.1514829	total: 10.4s	remaining: 2.25s
823:	learn: 0.1514328	total: 10.5s	remaining: 2.23s
824:	learn: 0.1513429	total: 10.5s	remaining: 2.22s
825:	learn: 0.1512975	total: 10.5s	remaining: 2.21s
826:	learn: 0.1511980	total: 10.5s	remaining: 2.19s
827:	learn: 0.1511458	total: 10.5s	remaining: 2.18s
828:	learn: 0.1511128	total: 10.5s	remaining: 2.17s
829:	learn: 0.1510668	total: 10.5s	remaining: 2.16s
830:	learn: 0.1509209	total: 10.5s	remaining: 2.14s
831:	learn: 0.1508754	total: 10.6s	remaining: 2.13s
832:	learn: 0.1508119	total: 10.6s	remaining: 2.12s
833:	learn: 0.1507693	total: 10.6s	remaining: 2.1s
834:	learn: 0.1506979	total: 10.6s	remaining: 2.09s
835:	learn: 0.1506048	total: 10.6s	remaining: 2.08s
836:	learn: 0.1505423	total: 10.6s	remaining: 2.07s
837:	learn: 0.1504781	total: 10.6s	remaining: 2.05s
838:	learn: 0.1504197	total: 10.6s	remaining: 2.04s
839:	learn: 0.1503713	total: 10.6s	remaining: 2.03s
840:	learn: 0

991:	learn: 0.1412059	total: 12.6s	remaining: 101ms
992:	learn: 0.1411713	total: 12.6s	remaining: 88.8ms
993:	learn: 0.1410749	total: 12.6s	remaining: 76.1ms
994:	learn: 0.1410570	total: 12.6s	remaining: 63.4ms
995:	learn: 0.1410089	total: 12.6s	remaining: 50.7ms
996:	learn: 0.1409684	total: 12.6s	remaining: 38ms
997:	learn: 0.1408953	total: 12.7s	remaining: 25.4ms
998:	learn: 0.1408580	total: 12.7s	remaining: 12.7ms
999:	learn: 0.1407701	total: 12.7s	remaining: 0us


In [247]:
from sklearn.metrics import f1_score
print(f1_score(y_test,cat))
#print(classification_report(y_test,cat))

0.9258649093904447


In [248]:
predictions = cat_model.predict(test_df)
sub= ttest[["ID"]]
sub['target'] = predictions
sub.to_csv('cat.csv', index = False)

In [249]:
sub['target'].unique()

array([1., 0.])

## MODEL ENSEMBLING

In [5]:
def model(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train, y_train)
        train_predict = model.predict(x_train)
        test_predict = model.predict(x_test)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test)
        
        print('\nValidation scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name
        
def Stacking(model,Train_stack,Test_stack,target,file_name):
    prediction = model.fit(Train_stack, target).predict(Test_stack)
    submission['target'] = prediction
    submission.to_csv(file_name,index=False)

## LGBM 1

In [6]:
def lgb(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=500,
                               verbose=250)
        train_predict = model.predict(x_train, num_iteration = model.best_iteration_)
        test_predict = model.predict(x_test, num_iteration = model.best_iteration_)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test, num_iteration = model.best_iteration_)
        
        print('\nValidation scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [7]:
lgb_model = lgbm.LGBMClassifier(random_state=34, n_estimators=8000, num_leaves=30,reg_lambda=6,reg_alpha=6 ,metric='auc', learning_rate=0.08,
    max_depth=8, class_weight = 'balanced')
LGB1_train, LGB1_test, LGB1_name =lgb(lgb_model,X, y, test_df.drop(['target'], axis =1),'lightgbm(1)')


NameError: name 'lgbm' is not defined

## LGBM 2

In [ ]:
lgb_model2 = lgbm.LGBMClassifier(random_state=34, n_estimators=100000,colsample_bytree=0.9,min_child_samples=10,
    subsample=0.5,subsample_freq=2,num_leaves=120,reg_lambda=2,reg_alpha=5 ,metric='auc', learning_rate=0.008,
    max_depth=5)
LGBM2_train, LGBM2_test, LGBM2_name= lgb(lgb_model2,X,y,test_df.drop(['target'], axis =1),'lightgbm(2)')

## CATBOOST 1

In [1832]:
def cat(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=500,
                           verbose=250,use_best_model=True)
        train_predict = model.predict(x_train)
        test_predict = model.predict(x_test)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test)
        print('\nTesting scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 8 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 8 folds split:',np.mean(mean_train))
    print('standard Deviation for 8 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [1833]:
catboost =  CatBoostClassifier(random_seed=34,use_best_model=True,
                          n_estimators=12000,silent=True,eval_metric='AUC', loss_function = 'CrossEntropy')

cat1_train, cat1_test, cat1_name = cat(catboost,X,y,test_df,'catboost(1)')

========================Fold1==========================
0:	test: 0.8398925	best: 0.8398925 (0)	total: 20ms	remaining: 4m
250:	test: 0.9687024	best: 0.9687024 (250)	total: 4.04s	remaining: 3m 8s
500:	test: 0.9718741	best: 0.9718741 (500)	total: 7.76s	remaining: 2m 58s
750:	test: 0.9731579	best: 0.9731751 (749)	total: 11.5s	remaining: 2m 52s
1000:	test: 0.9735622	best: 0.9735622 (1000)	total: 15s	remaining: 2m 44s
1250:	test: 0.9739903	best: 0.9739903 (1250)	total: 18.7s	remaining: 2m 40s
1500:	test: 0.9743798	best: 0.9743798 (1500)	total: 22.5s	remaining: 2m 37s
1750:	test: 0.9746000	best: 0.9746117 (1731)	total: 27.3s	remaining: 2m 39s
2000:	test: 0.9746170	best: 0.9746595 (1937)	total: 31.5s	remaining: 2m 37s
2250:	test: 0.9747382	best: 0.9747971 (2180)	total: 35.7s	remaining: 2m 34s
2500:	test: 0.9748137	best: 0.9748137 (2500)	total: 39.5s	remaining: 2m 30s
2750:	test: 0.9746930	best: 0.9748595 (2541)	total: 43.4s	remaining: 2m 25s
3000:	test: 0.9746956	best: 0.9748595 (2541)	total: 

In [1819]:
## CATBOOST 2

In [1820]:
catboost2 =  CatBoostClassifier(random_seed=34,bootstrap_type='Bayesian',max_depth=6,learning_rate=0.007,
                          iterations=12000,silent=True,eval_metric='AUC')


cat2_train, cat2_test, cat2_name = cat(catboost2,X,y,test_df,'catboost(2)')

========================Fold1==========================
0:	test: 0.7275302	best: 0.7275302 (0)	total: 6.81ms	remaining: 1m 21s
250:	test: 0.7914938	best: 0.7917202 (247)	total: 2.13s	remaining: 1m 39s
500:	test: 0.7997493	best: 0.7997720 (499)	total: 4.34s	remaining: 1m 39s
750:	test: 0.8045539	best: 0.8045539 (750)	total: 6.43s	remaining: 1m 36s
1000:	test: 0.8075860	best: 0.8076167 (998)	total: 8.28s	remaining: 1m 30s
1250:	test: 0.8104225	best: 0.8104241 (1247)	total: 10.1s	remaining: 1m 26s
1500:	test: 0.8128660	best: 0.8128660 (1500)	total: 12s	remaining: 1m 23s
1750:	test: 0.8153434	best: 0.8154227 (1743)	total: 13.7s	remaining: 1m 20s
2000:	test: 0.8173665	best: 0.8174053 (1996)	total: 15.4s	remaining: 1m 16s
2250:	test: 0.8181751	best: 0.8181751 (2250)	total: 17.1s	remaining: 1m 13s
2500:	test: 0.8187265	best: 0.8189755 (2470)	total: 18.9s	remaining: 1m 11s
2750:	test: 0.8190726	best: 0.8190726 (2750)	total: 20.7s	remaining: 1m 9s
3000:	test: 0.8198229	best: 0.8199474 (2974)	to

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:236: Feature Gender from pool must be 0.

## NAIVE BAYES

In [1881]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb_model = GaussianNB()
gnb_train, gnb_test, gnb_name = model(gnb_model,X,y,test_df.drop(['target'], axis =1),'GNB(1)')

========================Fold1==========================

Validation scores 0.16458852867830423

Training scores 0.18136952498457742
========================Fold2==========================

Validation scores 0.1724137931034483

Training scores 0.18953168044077132
========================Fold3==========================

Validation scores 0.17102137767220904

Training scores 0.18863361547762997
========================Fold4==========================

Validation scores 0.19902912621359223

Training scores 0.1788040260509177
========================Fold5==========================

Validation scores 0.19548872180451124

Training scores 0.16257088846880907
Average Testing ROC score for 10 folds split: 0.18050830949441302
Average Training ROC score for 10 folds split: 0.1801819470845411
standard Deviation for 10 folds split: 0.013974232695655666


In [1882]:
#predictions = cat_model.predict(test_df)
sub= ttest[["ID"]]
sub['target'] = cc['result']
sub.to_csv('gnb.csv', index = False)

In [1772]:
#from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression()
#model = lr.fit(X_train,y_train)
#pred = model.predict(X_test)
#sc = f1_score(y_test,pred)

In [1773]:
#preds = model.predict(test_df.drop(['target'], axis =1))

In [1774]:
#predictions = cat_model.predict(test_df)
#sub= ttest[["ID"]]
#sub['target'] = preds
#sub.to_csv('log.csv', index = False)

In [1792]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability = True))
mod = clf.fit(X_train, y_train)

In [1793]:
preds = mod.predict(X_test)
score = f1_score(y_test, preds)

In [1794]:
score

0.8084580530001373

In [1795]:
predi = mod.predict(test_df.drop(['target'], axis = 1))
sub= ttest[["ID"]]
sub['target'] = predi
sub.to_csv('logg.csv', index = False)

In [1775]:
#X[X==np.inf]=np.nan
#X.fillna(X.mean(), inplace=True)

## RANDOM FOREST

In [1776]:
rf1_model = RandomForestClassifier(n_jobs=-1,criterion='entropy',min_samples_split=10,n_estimators=2000,random_state=99)
rf1_train, rf1_test, rf1_name = model(rf1_model,X,y,test_df.drop(['target'], axis =1),'RandomForest(1)')

========================Fold1==========================

Validation scores 0.9306640625

Training scores 0.9681582438789047
========================Fold2==========================

Validation scores 0.9262796963017389

Training scores 0.968900675024108
========================Fold3==========================

Validation scores 0.92431640625

Training scores 0.970007227174175
========================Fold4==========================

Validation scores 0.9304369050524773

Training scores 0.968407090317135
========================Fold5==========================

Validation scores 0.936844672031212

Training scores 0.9683219694684124
Average Testing ROC score for 10 folds split: 0.9297083484270857
Average Training ROC score for 10 folds split: 0.968759041172547
standard Deviation for 10 folds split: 0.00431740650312337


In [1884]:
svc_model = SVC(gamma='auto', probability = True)
svc_train, svc_test, svc_name = model(svc_model,X,y,test_df.drop(['target'], axis =1),'SVC(1)')

========================Fold1==========================

Validation scores 0.07142857142857142

Training scores 0.09024979854955681
========================Fold2==========================


KeyboardInterrupt: 

## RANDOM FOREST 2

In [1777]:
rf_model2 = RandomForestClassifier(max_depth=10,min_samples_split=10,min_samples_leaf=15,n_estimators=4000,n_jobs=-1,random_state=34)
rf2_train, rf2_test, rf2_name= model( rf_model2,X,y,test_df.drop(['target'], axis =1),'RandomForest(2)')

========================Fold1==========================

Validation scores 0.8876404494382023

Training scores 0.9063030373968515
========================Fold2==========================

Validation scores 0.8899319408589532

Training scores 0.9076201641266121
========================Fold3==========================

Validation scores 0.8877957367064887

Training scores 0.9107865563962696
========================Fold4==========================

Validation scores 0.8954033771106943

Training scores 0.9087698646262508
========================Fold5==========================

Validation scores 0.9056427066260828

Training scores 0.907563025210084
Average Testing ROC score for 10 folds split: 0.8932828421480842
Average Training ROC score for 10 folds split: 0.9082085295512137
standard Deviation for 10 folds split: 0.006788760999053637


In [1778]:
def xgb(model,train,label,test,model_name):
    mean_train = []
    mean_test = []
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train.shape[0])
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train.iloc[train_index],train.iloc[test_index]
        y_train,y_test = label.iloc[train_index],label.iloc[test_index]
        print(f'========================Fold{count +1}==========================')
        model.fit(x_train, y_train, early_stopping_rounds = 500, eval_metric="auc",
                           eval_set=[(x_test, y_test)],verbose=250)
        train_predict = model.predict(x_train, ntree_limit = model.get_booster().best_ntree_limit)
        test_predict = model.predict(x_test, ntree_limit = model.get_booster().best_ntree_limit)
        val_pred[test_index] = test_predict
        test_pred+= model.predict(test, ntree_limit = model.get_booster().best_ntree_limit)
        
        print('\nTesting scores', f1_score(y_test,test_predict))
        print('\nTraining scores', f1_score(y_train,train_predict))
        mean_train.append(f1_score(y_train, train_predict))
        mean_test.append(f1_score(y_test,test_predict))
    print('Average Testing ROC score for 10 folds split:',np.mean(mean_test))
    print('Average Training ROC score for 10 folds split:',np.mean(mean_train))
    print('standard Deviation for 10 folds split:',np.std(mean_test))
    return val_pred, test_pred, model_name

In [1779]:
from xgboost import XGBClassifier
Xgboost = XGBClassifier(learning_rate=0.5,subsample=0.7,colsample_bytree=0.9,reg_alpha=6,
               n_jobs=-1,n_estimators=5000,max_depth= 7,random_state=34)

xgb_train, xgb_test, xgb_name= xgb(Xgboost,X,y,test_df.drop(['target'], axis =1),'xgboost')

========================Fold1==========================
[0]	validation_0-auc:0.89451
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.97347
[500]	validation_0-auc:0.97327
Stopping. Best iteration:
[141]	validation_0-auc:0.97382


Testing scores 0.9198067632850242

Training scores 0.9568955079307423
========================Fold2==========================
[0]	validation_0-auc:0.90063
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.97322
[500]	validation_0-auc:0.97260
[750]	validation_0-auc:0.97207
Stopping. Best iteration:
[281]	validation_0-auc:0.97346


Testing scores 0.921616358325219

Training scores 0.9664098242234529
========================Fold3==========================
[0]	validation_0-auc:0.88378
Will train until validation_0-auc hasn't improved in 500 rounds.
[250]	validation_0-auc:0.96869
[500]	validation_0-auc:0.96821
Stopping. Best iteration:
[166]	validation_0-auc:0.96930


Testing scores 0.

## DECISION TREE

In [1780]:
from sklearn.tree import DecisionTreeClassifier
dtc_model = DecisionTreeClassifier(max_depth=8,min_samples_leaf=5,criterion = 'entropy',min_samples_split=10,random_state=10)
dtc_train, dtc_test, dtc_name= model(dtc_model,X,y,test_df.drop(['target'], axis =1), 'dtc')

========================Fold1==========================

Validation scores 0.8188976377952756

Training scores 0.8207000463606862
========================Fold2==========================

Validation scores 0.8103254769921436

Training scores 0.8200135074290861
========================Fold3==========================

Validation scores 0.8057869355545814

Training scores 0.8240551724137931
========================Fold4==========================

Validation scores 0.8098720292504571

Training scores 0.8113314447592068
========================Fold5==========================

Validation scores 0.8059836808703535

Training scores 0.8101941196503575
Average Testing ROC score for 10 folds split: 0.8101731520925621
Average Training ROC score for 10 folds split: 0.8172588581226259
standard Deviation for 10 folds split: 0.004754389408908176


## GBM

In [1781]:
gbm_model = GradientBoostingClassifier(max_depth=4,min_samples_leaf=10,n_estimators=2000,learning_rate=0.1,min_samples_split=10,random_state=10)
gbm_train, gbm_test, gbm_name= model(gbm_model,X,y,test_df.drop(['target'], axis =1), 'Gbm')

========================Fold1==========================

Validation scores 0.9241912119748914

Training scores 0.9781013186944328
========================Fold2==========================

Validation scores 0.9216969696969698

Training scores 0.9796040076335878
========================Fold3==========================

Validation scores 0.9140926640926641

Training scores 0.9807589205933163
========================Fold4==========================

Validation scores 0.9220716360116166

Training scores 0.9794847328244275
========================Fold5==========================

Validation scores 0.9298414223930803

Training scores 0.978982565082398
Average Testing ROC score for 10 folds split: 0.9223787808338445
Average Training ROC score for 10 folds split: 0.9793863089656325
standard Deviation for 10 folds split: 0.005063494064365597


In [1782]:
train_stack = pd.DataFrame(xgb_train)
Stack_Train = pd.concat([train_stack,pd.DataFrame(rf1_train),pd.DataFrame(rf2_train),
                        pd.DataFrame(LGB1_train),pd.DataFrame(LGBM2_train),pd.DataFrame(cat1_train),pd.DataFrame(cat2_train),
                         pd.DataFrame(gbm_train)],1)
test_stack = pd.DataFrame(xgb_test)
Stack_Test = pd.concat([test_stack,pd.DataFrame(rf1_test),pd.DataFrame(rf2_test),
                        pd.DataFrame(LGB1_test),pd.DataFrame(LGBM2_test),pd.DataFrame(cat1_test),pd.DataFrame(cat2_test),
                        pd.DataFrame(gbm_test)],1)

Stack_Train.columns=[xgb_name,rf1_name,rf2_name,LGB1_name,LGBM2_name,cat1_name,cat2_name,gbm_name]
Stack_Test.columns=[xgb_name,rf1_name,rf2_name,LGB1_name,LGBM2_name,cat1_name,cat2_name,gbm_name]

Stack_Test = Stack_Test/5
Stack_Test.head()

,xgboost,RandomForest(1),RandomForest(2),lightgbm(1),lightgbm(2),catboost(1),catboost(2),Gbm
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.2,0.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.8,1.0,1.0,1.0,1.0,1.0,1.0


In [1784]:
from sklearn.linear_model import LogisticRegression, LinearRegression
base_model = LogisticRegression()
Stacking(base_model,Stack_Train,Stack_Test,y,'final_stack11.csv')

## Tried logistic regression but it didnt give a good score